In [1]:
## Instalação / Atualização das bibliotecas necessárias
!pip install --upgrade pip
!pip install transformers
!pip install unidecode
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1.1
    Uninstalling pip-22.1.1:
      Successfully uninstalled pip-22.1.1
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-6i1or1ai/farm-haystack_7553af90962440a489b975ee5dca1984
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-6i1or1ai/farm-haystack_7553af90962440a489b975ee5dca1984
  Resolved https://github.com/deepset-ai/haystack.git to commit 2a7e333d9a5d74c4fcdadeed372dd926734b2402
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 62.5 MB/s eta 0:00:00
     ━━

In [2]:
## Importação das Bibliotecas básicas
import pandas as pd
import numpy as np
from tqdm import tqdm
import string
import unidecode
import os, glob, re, sys, random, unicodedata, collections
import pathlib
from pathlib import Path

## Importação do BERT - Hugginface
from transformers import AutoModel, AutoTokenizer, AutoModelForPreTraining, BertTokenizer, BertForSequenceClassification, pipeline

## Importação das bibliotecas da Haystack
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader, DensePassageRetriever, PreProcessor
from haystack.document_stores import FAISSDocumentStore
from haystack.pipelines import ExtractiveQAPipeline

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
files = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/livros/memorial-de-aires.txt
/kaggle/input/livros/esau.txt
/kaggle/input/livros/ressurreicao.txt
/kaggle/input/livros/memoriasBras.txt
/kaggle/input/livros/maoLuva.txt
/kaggle/input/livros/quincas.txt
/kaggle/input/livros/casaVelha.txt
/kaggle/input/livros/helena.txt
/kaggle/input/livros/domCasmurro.txt
/kaggle/input/livros/iaia.txt


In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [5]:
# Apontando ao diretório dos livros.
doc_dir = dirname
# Convertendo arquivos em dicionários
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Escrevendo o dicionário no BD local.
document_store.write_documents(docs)

# apontando para o modelo treinado
nlp = pipeline("question-answering", model='pierreguillou/bert-base-cased-squad-v1.1-portuguese')

Writing Documents:   0%|          | 0/10 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

2022-07-13 23:46:39.475864: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
perguntas = ['Quem é Capitu?', 'Quem foi Mascarenhas?', 'Quem é a filha de Dona Eusébia e do Vilaça?', 'Rubião e Cristiano viram sócios em que?', 'Por que Estevão desejava morrer?', 'Salvador era pai de quem?', 'Com quem Jorge se casa ao voltar do Paraguai?', 'Por quem Pedro e Paulo estavam apaixonados?', 'Quem escreve sobre Tristão e Fidélia?', 'Como Escobar morreu?']

resultados = pd.DataFrame(columns=['Pergunta','Documento','Score'])
i = 0

for pergunta in tqdm(perguntas, desc='Perguntas'):
  for documento in tqdm(docs, desc='Livros'):
      result = nlp(question=pergunta, context=documento.content)
      resultados.loc[i] = [pergunta, documento.meta['name'], result['score']]
      i += 1

#resultados.sort_values(by='Score', ascending=False).head()
top3_results = resultados.sort_values(by=['Pergunta','Score'], ascending=False).groupby(by='Pergunta').head(3)
top3_results

Perguntas: 100%|██████████| 10/10 [6:24:34<00:00, 2307.49s/it]


,Pergunta,Documento,Score
51,Salvador era pai de quem?,esau.txt,0.993063
54,Salvador era pai de quem?,maoLuva.txt,0.982218
58,Salvador era pai de quem?,domCasmurro.txt,0.976742
38,Rubião e Cristiano viram sócios em que?,domCasmurro.txt,0.949915
35,Rubião e Cristiano viram sócios em que?,quincas.txt,0.918761
30,Rubião e Cristiano viram sócios em que?,memorial-de-aires.txt,0.902077
23,Quem é a filha de Dona Eusébia e do Vilaça?,memoriasBras.txt,0.995128
28,Quem é a filha de Dona Eusébia e do Vilaça?,domCasmurro.txt,0.893009
25,Quem é a filha de Dona Eusébia e do Vilaça?,quincas.txt,0.847360
8,Quem é Capitu?,domCasmurro.txt,0.840419
